In [132]:
import geopandas as gpd
import osmnx as ox
import xgboost as xgb
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import time 
import warnings
import time
import pandas as pd
import matplotlib.pyplot as plt

In [139]:
path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm filtre\batiment"
list_departement=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Liste-Excel-des-departements-francais.xlsx")

In [136]:
iris_data=pd.read_excel("Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - IRIS\Geodata_IRIS_essentiels.xlsx")
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp').to_crs('EPSG:3857')

In [140]:
building_no_na=building[~building['building:levels'].isna()]
building_na=building[building['building:levels'].isna()]

#get contours of the departement only 
dep_contour_iris=contour_iris[contour_iris['INSEE_COM'].isin([str(i) for i in range(dep*1000,dep*1000+1000)])]

In [82]:
a=gpd.sjoin(building,dep_contour_iris,predicate="intersects",how="left")
a = a[~a.index.duplicated(keep='first')].reset_index()

In [83]:
iris_building_id=a[["index_right",'element_type',"osmid"]].copy()
iris_building_id=iris_building_id.set_index(['element_type',"osmid"])

In [84]:
a=gpd.GeoDataFrame(
    a[['element_type',"osmid","CODE_IRIS"]].merge(contour_iris[['CODE_IRIS',"geometry"]]),

).set_geometry("geometry").set_crs('EPSG:3857')

xx=gpd.sjoin(a,dep_contour_iris,predicate="touches")

In [85]:
nearby_buildings=xx[["osmid",'element_type',"CODE_IRIS_left","index_right"]].groupby(["element_type",'osmid'])['index_right'].apply(list)

In [86]:
nearby_buildings=nearby_buildings.to_dict()

# ML

In [87]:
sample=building_no_na.sample(frac=1,random_state=123)

In [88]:
def f (x):
    if x!=x:
        return -100
    else:
        return float(x.split(';')[0])

In [89]:
data_no=None
#pour l'affichage
j=0

start=time.time()
warnings.filterwarnings("ignore")
k=10
#y_true
building_level=[]

print('number of rows :'+str(len(sample)))
for i ,row in sample.iterrows():
    if j%1000==0:
        print(j)
    building_level.append(row['building:levels'])
        
    #get the distance between all buildings and the current buildingi
    ## get the com nearby of the building i
    if i in nearby_buildings.keys() and nearby_buildings[i][0] is not None:
        current_iris_index=iris_building_id.loc[i]['index_right']
        iris_ids=list(nearby_buildings[i])
        iris_ids.append(current_iris_index)
        indices=list(iris_building_id[iris_building_id['index_right'].isin(iris_ids)].index)
        
    else:
        #indices=building.index
        indices=list(building_no_na.index)
        
    
    #considerer que les batiment dont on connait leurs nombre d'étages

    indices = building_no_na.index.intersection(indices)
    distance=building_no_na.loc[indices].distance(row['geometry']).sort_values()
    
    #remove the current building from the distance dataframe (distance will be 0 ofc)
    distance=distance.drop(i)

    #take the k nearest neighboors
    a=building_no_na.loc[distance.index].iloc[:k]

    #buildings nearby
    areas=a.area.to_numpy()
    distances=distance.iloc[:k].to_numpy()
    types=a['building'].to_numpy()
    levels=np.vectorize (f )(a['building:levels'].to_numpy())
    
    if len(areas)<k:
        areas=np.append(areas,[np.nan for _ in range(k-len(a))])
    if len(distances)<k:distances=np.append(distances,[np.nan for _ in range(k-len(a))])
    if len(types)<k:types=np.append(types,[np.nan for _ in range(k-len(a))])
    if len(levels)<k:levels=np.append(levels,[np.nan for _ in range(k-len(a))])
    
    #information about the iris
    iris=contour_iris[contour_iris.intersects(row['geometry'])].iloc[0]
    if iris['TYP_IRIS'] ==iris['TYP_IRIS']:
        iris_type=np.array([iris['TYP_IRIS']])
    else:
        iris_type=None
        
    iris_area=np.array([iris['geometry'].area])
    
    #information about the building itself
    type_building=np.array([row['building']])
    area_building=np.array([row['geometry'].area])
    
    #densité 
    area=iris["geometry"].area/10**6
    iris=iris_data[iris_data['CODE_IRIS']==iris["CODE_IRIS"]]
    pop=iris['P17_POP'].iloc[0]
    densite=[pop/area]

    

    if data_no is None:
        
        data_no=np.concatenate([type_building,area_building,iris_type,iris_area,densite,areas,types,levels,distances])
    else:
        
        data_no=np.vstack([data_no,np.concatenate([type_building,area_building,iris_type,iris_area,densite,areas,types,levels,distances])])

        
            
    j+=1
end=time.time()
print(end-start)

number of rows :58625
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
2955.161912918091


In [90]:
#create a dataframe

columns=["type_building","area_building","iris_type","iris_area","iris_densite"]
for j in ["areas","types","levels","distances"]:
    for i in range(k):
        columns.append(j+str(i))

data_no=pd.DataFrame(data_no,columns=columns)
data_no.head()

,type_building,area_building,iris_type,iris_area,iris_densite,areas0,areas1,areas2,areas3,areas4,...,distances0,distances1,distances2,distances3,distances4,distances5,distances6,distances7,distances8,distances9
0,apartments,524.472914,H,95817.583049,22640.846933,447.019561,220.198506,435.068407,586.947648,690.433694,...,0.0,22.733941,31.053644,31.863299,40.188677,49.75681,54.952447,56.857179,66.0365,67.889794
1,apartments,315.317166,H,113245.430628,18981.442776,229.96435,506.046641,336.842886,216.908724,272.477788,...,0.0,0.0,13.111436,13.991268,15.145654,15.148969,15.539586,17.19743,19.93448,21.371818
2,yes,547.87119,H,323613.170988,13913.789769,404.249459,372.147857,36.326792,404.095655,1610.150961,...,0.0,0.0,8.280352,12.538186,12.801741,13.582031,26.460503,34.300625,35.516815,43.068984
3,yes,436.288912,H,103590.242518,15277.502621,925.280686,68.784896,464.310302,375.911824,372.737956,...,0.0,2.312798,19.794707,19.851513,27.793037,36.804469,39.865849,40.444343,43.746732,47.781417
4,yes,74.652713,H,247002.138863,12007.645378,3967.225881,241.74481,617.659218,218.862945,126.634351,...,23.798155,45.442932,80.660596,82.761556,87.588175,91.793512,94.672215,104.409465,115.789509,116.832409


In [91]:
#building_level is the true label Y
building_level=np.array([float(i.split(';')[0]) for i in building_level])
building_level=building_level[~data_no.isna().any(axis=1)]
building_level.shape

(58625,)

In [93]:
#convert columns from object to int
for c in data_no:
    if "area" in c or "levels" in c or "distances" in c or "iris_densite" in c:
        data_no[c]=data_no[c].astype(np.int)

In [94]:
list_encoders=[]
from sklearn.preprocessing import OrdinalEncoder
for c in data_no.columns:
    if data_no[c].dtype=="O":
        print(c)
        enc = OrdinalEncoder()
        data_no[c]=enc.fit_transform(data_no[c].to_numpy().reshape(-1,1) ).squeeze().astype(np.int)
        list_encoders.append(enc)
        

type_building
iris_type
types0
types1
types2
types3
types4
types5
types6
types7
types8
types9


In [95]:
X_train, X_test, y_train, y_test =train_test_split(data_no,building_level,random_state=123)

# XGBoost

In [96]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [97]:
param = {'max_depth': 5, 'eta': 0.1, 'objective': 'reg:squarederror',"gamma":2,"lambda":4,"verbosity":0}
param['nthread'] = 4
param['eval_metric'] = ["mae"]
param['verbosity']=0
evallist = [(dtrain, 'train'), (dtest, 'eval')]
num_round = 500

In [98]:
start=time.time()
model = xgb.train(param, dtrain, num_round)
### ADDD Other metrics STD and mediane absolute error
end=time.time()
ypred = model.predict(dtest)
print('The error is :'+str(np.sum(np.abs(model.predict(dtest).round()-y_test))/len(y_test)))
print('Train error is :'+str(np.sum(np.abs(model.predict(dtrain).round()-y_train))/len(y_train)))

print("Std train error:",np.std(np.abs(model.predict(dtrain).round()-y_train)))
print("Std test error:",np.std(np.abs(model.predict(dtest).round()-y_test)))

print("Median Abs error ",np.std(np.abs(model.predict(dtest).round()-y_test)))
print('Train time '+str(end-start))

The error is :1.170430511018626
Train error is :0.9168258733624454
Std train error: 1.0162976900441545
Std test error: 1.4044895456264517
Median Abs error  1.4044895456264517
Train time 12.569924116134644


In [100]:
pickle.dump(model, open(place_name.lower()+".model.pkl","wb"))

# Stats

In [101]:
intervals=[5,10,15,20]

In [102]:
ypred=model.predict(dtest)

In [103]:
for i,el in enumerate(intervals):
    
    if i ==0:
        a="0"
        b=str(el)
        mask=y_test<=el
    elif i ==len(intervals)-1:
        a="+infini"
        mask=y_test>el
    else:
        a=str(intervals[i-1])
        b=str(el)
        mask=np.logical_and(y_test<=el , y_test>intervals[i-1])

    print('For intervale ['+a+","+b+"] The error is :"+str(np.sum(np.abs(ypred[mask].round()-y_test[mask]))/len(y_test[mask])))
        

For intervale [0,5] The error is :1.3916999201915403
For intervale [5,10] The error is :0.8746166114587167
For intervale [10,15] The error is :4.461928934010152
For intervale [+infini,15] The error is :12.428571428571429


In [104]:
da=pd.DataFrame([],columns=["level","error",'frequency'])
for i,el in enumerate(pd.Series(y_test).value_counts().index):
    mask=y_test==el
    da.loc[len(da)]=[el,np.sum(np.abs(ypred[mask].round()-y_test[mask]))/len(y_test[mask]),pd.Series(y_test).value_counts()[el]]

In [105]:
da.sort_values('error').head(10)

,level,error,frequency
1,6.0,0.669509,2463.0
0,7.0,0.694388,3956.0
2,5.0,0.895136,1583.0
5,4.0,1.272040,1191.0
4,8.0,1.281432,1201.0
7,3.0,1.467118,1034.0
23,2.5,1.500000,2.0
6,2.0,1.645777,1101.0
3,1.0,1.809382,1343.0
8,9.0,1.967005,394.0


In [106]:
da.sort_values('error').corr()

,level,error,frequency
level,1.000000,0.785100,-0.455676
error,0.785100,1.000000,-0.458828
frequency,-0.455676,-0.458828,1.000000


**Conclusion**
L'erreur est trés bas pour lorsque le nombre d'étage est dans l'intervale [5-7] qui correpond au nombre d'étage moyen à Paris.
Les erreurs apparaissent pour les buildings ayant un trés grand nombre d'étage.

# Prediction des valeurs manquantes

In [107]:
data_no_pred=None
j=0
import warnings
warnings.filterwarnings("ignore")
k=10
#the value to predict
building_level=[]
print('number of rows :'+str(len(building_na)))
for i ,row in building_na.iterrows():
    if j%1000==0:
        print(j)

        
    #get the distance between all buildings and our building
    ## get com nearby of the building
    if i in nearby_buildings.keys() and nearby_buildings[i][0] is not None:
        current_iris_index=iris_building_id.loc[i]['index_right']
        iris_ids=list(nearby_buildings[i])
        iris_ids.append(current_iris_index)
        indices=list(iris_building_id[iris_building_id['index_right'].isin(iris_ids)].index)
        
    else:
        #indices=building.index
        indices=list(building_no_na.index)
    
    #considerer que les batiment dont on connait leurs nombre d'étages

    indices = building_no_na.index.intersection(indices)
    distance=building_no_na.loc[indices].distance(row['geometry']).sort_values()


    #take the k nearest neighboors
    a=building_no_na.loc[distance.index].iloc[:k]
    
    
    
    
    #buildings nearby
    areas=a.area.to_numpy()
    distances=distance.iloc[:k].to_numpy()
    types=a['building'].to_numpy()
    levels=np.vectorize (f )(a['building:levels'].to_numpy())
    if len(areas)<k:
        areas=np.append(areas,[np.nan for _ in range(k-len(a))])
    if len(distances)<k:distances=np.append(distances,[np.nan for _ in range(k-len(a))])
    if len(types)<k:types=np.append(types,[np.nan for _ in range(k-len(a))])
    if len(levels)<k:levels=np.append(levels,[np.nan for _ in range(k-len(a))])
    
    #information about the iris
    iris=contour_iris[contour_iris.intersects(row['geometry'])].iloc[0]
    if iris['TYP_IRIS'] ==iris['TYP_IRIS']:
        iris_type=np.array([iris['TYP_IRIS']])
    else:
        iris_type=None
        
    iris_area=np.array([iris['geometry'].area])
    
    #densité 
    area=iris["geometry"].area/10**6
    iris=iris_data[iris_data['CODE_IRIS']==iris["CODE_IRIS"]]
    pop=iris['P17_POP'].iloc[0]
    densite=[pop/area]

    #information about the building itself
    type_building=np.array([row['building']])
    area_building=np.array([row['geometry'].area])

    if data_no_pred is None:
        
        data_no_pred=np.concatenate([type_building,area_building,iris_type,iris_area,densite,areas,types,levels,distances])
    else:
    
        data_no_pred=np.vstack([data_no_pred,np.concatenate([type_building,area_building,iris_type,iris_area,densite,areas,types,levels,distances])])

        
            
    j+=1
    

number of rows :34917
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000


In [108]:
#create a dataframe
columns=["type_building","area_building","iris_type","iris_area","iris_densite"]
for j in ["areas","types","levels","distances"]:
    for i in range(k):
        columns.append(j+str(i))

data_no_pred=pd.DataFrame(data_no_pred,columns=columns)
data_no_pred.head()

,type_building,area_building,iris_type,iris_area,iris_densite,areas0,areas1,areas2,areas3,areas4,...,distances0,distances1,distances2,distances3,distances4,distances5,distances6,distances7,distances8,distances9
0,tower,22541.645496,D,634132.521678,0.0,45.33129,84.284943,829.063573,1577.863334,771.138474,...,9.063766,92.598661,129.539554,129.680164,139.985452,154.729647,172.136503,188.776462,188.965767,194.235301
1,yes,4133.14538,H,190461.466524,16711.021643,646.271123,70.621419,269.381765,238.106493,217.060702,...,7.65226,7.95732,13.490055,20.338188,21.509935,21.58861,23.444724,26.815201,32.391041,40.297783
2,yes,7583.947859,H,269819.368861,16651.734306,329.18327,136.613253,2436.999333,522.035164,351.205214,...,9.305743,11.151359,12.838446,22.697683,23.115129,26.828584,27.02731,27.172772,28.682577,29.999008
3,church,3695.142465,H,116038.103453,15022.211203,499.193999,838.442338,370.745416,790.170421,513.089647,...,16.963922,21.328591,29.532356,37.998404,51.471611,60.054584,60.162066,60.613656,63.931068,87.103173
4,yes,8109.101385,H,239523.351589,10080.484602,3482.801258,3589.270482,784.506622,171.798862,5613.529801,...,22.599454,22.741726,73.917944,74.692231,74.924919,75.6431,76.682008,79.534656,85.013466,91.246005


In [109]:
data_no_pred_backup=data_no_pred.copy()

In [110]:
#convert columns from object to int
for c in data_no_pred:
    if "area" in c or "levels" in c or "distances" in c or "iris_densite" in c:
        data_no_pred[c]=data_no_pred[c].astype(np.int)

In [111]:
i=0
for c in data_no_pred.columns:
    if data_no_pred[c].dtype=="O":

        
        enc=list_encoders[i]
        enc.set_params(encoded_missing_value=-1,handle_unknown="use_encoded_value")

        data_no_pred[c]=enc.transform(data_no_pred[c].to_numpy().reshape(-1,1) ).squeeze().astype(np.int)
        i+=1
        

In [112]:
data_no_pred=xgb.DMatrix(data_no_pred)

In [113]:
prediction=model.predict(data_no_pred).round()

In [116]:
pd.Series(prediction).value_counts()

 6.0     10081
 7.0      6493
 5.0      6073
 4.0      3038
 2.0      2630
 3.0      2269
 1.0      2137
 8.0      1102
 0.0       404
 9.0       330
 10.0      147
 11.0       67
 12.0       43
-1.0        18
 15.0       17
 14.0       15
 13.0       15
 18.0       11
 16.0        9
 20.0        6
 17.0        3
 28.0        2
 21.0        2
-2.0         1
 23.0        1
 22.0        1
 32.0        1
 25.0        1
dtype: int64

In [114]:


j=0
for i , row in building.loc[building['building:levels'].isna()].iterrows():
    building.loc[i,"building:levels"]=str(prediction[j])
    j+=1

In [119]:
pickle.dump(building,open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+place_name.lower()+"\\processed data\\building_level_filled.pkl",'wb'))